Importando as bibliotecas

In [160]:
import numpy as np
import pandas as pd

Carregando base de dados

In [161]:
# Treinamento com a variável alvo
transact_train_database = pd.read_csv('data/transact_train.txt', sep='|')

# Teste sem a variável alvo
transact_test_database = pd.read_csv('data/transact_class.txt', sep = '|')

# Variável alvo do teste
transact_original_database = pd.read_csv('data/realclass_t1.txt', sep = '|')

Alterando a granularidade

In [162]:
transact_train_database = transact_train_database.drop_duplicates(subset=['sessionNo'], keep='last')
transact_train_database_y = transact_train_database['order']
transact_train_database_y = transact_train_database_y.replace(to_replace=['y', 'n'], value=[1, 0])
transact_train_database_x = transact_train_database.drop(['order'], axis=1)

transact_test_database_x = transact_test_database.drop_duplicates(subset=['sessionNo'], keep='last')

In [163]:
transact_train_database_x.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,...,y,completely orderable,1,600,70,21,1,43,1,49
7,2,6,5,94.469,0,?,?,?,0,?,...,y,completely orderable,?,?,?,?,?,?,?,?
12,3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,...,y,completely orderable,3,1800,475,302,12,45,1,11
14,4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,...,y,completely not orderable,?,?,?,?,?,?,?,?
22,5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,...,y,completely orderable,4,800,503,18,1,46,1,40


In [164]:
transact_test_database_x.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
8,1,18,7,624.606,11,16.99,39.99,207.91,5,16.99,...,y,completely orderable,25039,1300,489,188,5,49,1,65
19,2,18,7,2804.705,16,34.99,34.99,174.95,2,34.99,...,y,completely orderable,25040,1200,543,43,5,29,2,184
42,3,18,7,7401.384,119,7.99,59.95,3263.57,12,12.49,...,y,completely orderable,25041,600,552,17,4,37,2,107
48,4,18,7,2853.550,152,3.99,239.99,5642.5,4,9.99,...,?,?,25042,8500,535,226,19,49,2,17
50,5,18,7,48.145,2,29.99,29.99,59.98,1,29.99,...,y,completely orderable,25043,600,543,39,2,53,2,234


In [165]:
transact_train_database_y.head()

2     1
7     1
12    1
14    0
22    1
Name: order, dtype: int64

Tratando os valores ausentes

Lista de variáveis numéricas

In [166]:
list_numeric_variables = [
  'cMinPrice',
  'cMaxPrice',
  'cSumPrice',
  'bMinPrice',
  'bMaxPrice',
  'bSumPrice',
  'bStep',
  'maxVal',
  'customerScore',
  'accountLifetime',
  'payments',
  'age',
  'address',
  'lastOrder',
]

Lista de variáveis string

In [167]:
list_string_variables = [
  'availability',
  'onlineStatus',
]

In [168]:
def replaceValueMissing(df, columns, value):
  removed = df[columns].replace('?', value)
  for column in columns:
    df[column] = removed[column]

  return df

In [169]:
def convertFloat(df, columns):
  for column in columns:
    df[column] = df[column].astype(float)

  return df

In [170]:
def replaceMissingByMean(df, columns):
  for column in columns:
    df[column] = df[column].replace(to_replace='?', value=df[column].mean())

  return df

In [171]:
def replaceMissingByMode(df, columns):
  for column in columns:
    df[column] = df[column].replace(to_replace='?', value=df[column].mode()[0])

  return df

In [172]:
def replaceMissingByFrequency(df, columns):
  for column in columns:
    df[column] = df[column].replace(to_replace='?', value=df[column].value_counts().index[0])

  return df

In [173]:
def replaceMissingByMedian(df, columns):
  for column in columns:
    df[column] = df[column].replace(to_replace='?', value=df[column].median())

  return df

In [174]:
def replaceMissingByFixedValue(df, columns, value):
  for column in columns:
    df[column] = df[column].replace(to_replace='?', value=value)

  return df

In [175]:
transact_train_database_x.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,...,y,completely orderable,1,600,70,21,1,43,1,49
7,2,6,5,94.469,0,?,?,?,0,?,...,y,completely orderable,?,?,?,?,?,?,?,?
12,3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,...,y,completely orderable,3,1800,475,302,12,45,1,11
14,4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,...,y,completely not orderable,?,?,?,?,?,?,?,?
22,5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,...,y,completely orderable,4,800,503,18,1,46,1,40


In [176]:
transact_test_database_x.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
8,1,18,7,624.606,11,16.99,39.99,207.91,5,16.99,...,y,completely orderable,25039,1300,489,188,5,49,1,65
19,2,18,7,2804.705,16,34.99,34.99,174.95,2,34.99,...,y,completely orderable,25040,1200,543,43,5,29,2,184
42,3,18,7,7401.384,119,7.99,59.95,3263.57,12,12.49,...,y,completely orderable,25041,600,552,17,4,37,2,107
48,4,18,7,2853.550,152,3.99,239.99,5642.5,4,9.99,...,?,?,25042,8500,535,226,19,49,2,17
50,5,18,7,48.145,2,29.99,29.99,59.98,1,29.99,...,y,completely orderable,25043,600,543,39,2,53,2,234


Base na granularidade do projeto

In [177]:
# Substituindo a string ? por np.nan em todas p/ todas as variáveis da base na granularidade do projeto

transact_train_database_x = replaceValueMissing(transact_train_database_x, list_numeric_variables, np.nan)
transact_train_database_x = replaceValueMissing(transact_train_database_x, list_string_variables, np.nan)

transact_test_database_x = replaceValueMissing(transact_test_database_x, list_numeric_variables, np.nan)
transact_test_database_x = replaceValueMissing(transact_test_database_x, list_string_variables, np.nan)

transact_train_database_x = convertFloat(transact_train_database_x, list_numeric_variables)
transact_test_database_x = convertFloat(transact_test_database_x, list_numeric_variables)

# Substituindo os missing values das variáveis numéricas pela média
transact_train_database_x = replaceMissingByMean(transact_train_database_x, list_numeric_variables)
transact_test_database_x = replaceMissingByMean(transact_test_database_x, list_numeric_variables)

# Substituindo os missing values das variáveis String por um valor fixo
transact_train_database_x = replaceMissingByFixedValue(transact_train_database_x, list_string_variables, 'ausente')
transact_test_database_x = replaceMissingByFixedValue(transact_test_database_x, list_string_variables, 'ausente')

/home/uendel/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [178]:
transact_train_database_x.sample(3)

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
381862,44872,14,7,272.437,2,89.99,89.99,179.98,1,89.99,...,y,completely orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413685,48305,17,7,1064.585,50,14.99,49.99,1352.97,2,17.99,...,y,completely orderable,15848,4000.0,543.0,209.0,30.0,48.0,2.0,10.0
86865,10435,17,5,1338.443,9,34.99,69.95,349.87,1,34.99,...,NaN,NaN,5147,300.0,495.0,0.0,0.0,43.0,2.0,29.0


In [181]:
# Atribuindo valor zero para as variáveis comportamentais que não possuem valor de customerNo

rows_without_customerNo = transact_train_database_x['customerNo'] == '?'
transact_train_database_x.loc[rows_without_customerNo, 'maxVal'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'customerScore'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'accountLifetime'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'payments'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'age'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'address'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'lastOrder'] = 0

# Teste
rows_test_without_customerNo = transact_test_database_x['customerNo'] == '?'
transact_test_database_x.loc[rows_test_without_customerNo, 'maxVal'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'customerScore'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'accountLifetime'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'payments'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'age'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'address'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'lastOrder'] = 0

In [197]:
# Substituindo a string ? por np.nan em todas p/ todas as variáveis da base a base original
transact_test_database = replaceValueMissing(transact_test_database.copy(), list_numeric_variables, np.nan)
transact_test_database = replaceValueMissing(transact_test_database.copy(), list_string_variables, np.nan)
transact_train_database = replaceValueMissing(transact_train_database.copy(), list_numeric_variables, np.nan)
transact_train_database = replaceValueMissing(transact_train_database.copy(), list_string_variables, np.nan)

transact_train_database = convertFloat(transact_train_database, list_numeric_variables)
transact_test_database = convertFloat(transact_test_database, list_numeric_variables)

In [ ]:
transact_train_database_x.describe(include='all')

In [ ]:
transact_test_database_x.describe(include='all')

In [ ]:
transact_train_database.head()

In [199]:
transact_test_database.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
0,1,18,7,136.833,3,39.99,39.99,79.98,1,39.99,...,y,completely orderable,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
1,1,18,7,189.984,3,39.99,39.99,79.98,1,39.99,...,y,completely orderable,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
2,1,18,7,342.894,6,16.99,39.99,113.96,2,16.99,...,NaN,NaN,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
3,1,18,7,411.051,8,16.99,39.99,149.94,3,16.99,...,NaN,NaN,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
4,1,18,7,460.049,10,16.99,39.99,189.92,4,16.99,...,NaN,NaN,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
